In [1]:
import pandas as pd
import json
import nltk
from nltk.probability import FreqDist
from funciones import get_sent_tokens,get_lemma,get_stems,remove_stop_words,get_word_tokens, remove_special_characters

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data2 = pd.read_csv("../data/booksummaries.txt", delimiter="\t", header=None)
data2.columns = ["id","random", "book_name", "author", "date_of_publish", "genres", "summary"]
data2

,id,random,book_name,author,date_of_publish,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [3]:
#necesitamos que esté el género y quitamos columnas que no son útiles
data2 = data2.dropna(subset="genres").drop(columns=["id", "random", 'book_name', 'author', 'date_of_publish'])

In [4]:
def fix_genre(row):
    return json.loads(row.genres).values()

data2.loc[:,"genres"] = data2.apply(fix_genre, axis=1)
data2

,genres,summary
0,"(Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,"(Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,"(Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
4,"(Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...
5,"(War novel, Roman à clef)","The book tells the story of Paul Bäumer, a Ge..."
...,...,...
16549,(Science Fiction),The story starts with former government agent...
16551,"(Thriller, Fiction, Suspense)",The series follows the character of Nick Ston...
16555,"(Thriller, Fiction)",The reader first meets Rapp while he is doing...
16556,(Autobiography),The book follows very rough chronological ord...


In [5]:
# 'Crime Fiction'
# 'Fantasy'
# 'Historical novel'
# 'Horror'
# 'Science Fiction'
# 'Thriller'
# 

In [6]:
initial_genres = list(data2.explode("genres").genres.value_counts().index)

In [7]:
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)

In [8]:
data2.explode("genres").genres.value_counts()

genres
Fiction                4747
Speculative fiction    4314
Science Fiction        2870
Novel                  2463
Fantasy                2413
                       ... 
Fable                     1
Bangsian fantasy          1
Space western             1
Fantastique               1
Pastiche                  1
Name: count, Length: 227, dtype: int64

In [9]:
def map_genres(row):
    r = row.lower()
    #main 5 from other dataset
    if "science fiction" in r:
        return "Science Fiction"
    if "fantasy" in r:
        return "Fantasy"
    # if "historical novel" in r:
    #     return "Historical novel"
    # if "horror" in r:
    #     return "Horror"
    if "thriller" in r:
        return "Thriller"
    
    if "crime fiction" in r:
        return "Crime Fiction"
    if "speculative fiction" in r:
        return None
        # return "Speculative fiction"
    if "children's literature" in r:
        return "Children's literature"
    if "mystery" in r:
        return "Mystery"
    # if "young adult literature" in r:
    #     return "Young adult literature"
    if "suspense" in r:
        return "Thriller"
    # if "historical fiction" in r:
    #     return "Historical Fiction"
    if "detective fiction" in r:
        return "Crime Fiction"
    
    
    #fillers
    # if "novel" in r:
    #     return "Novel"
    if "fiction" in r:
        return "Fiction"

    
    

In [10]:
data2.explode("genres").genres.apply(map_genres).value_counts()

genres
Fiction                  5918
Science Fiction          2953
Fantasy                  2594
Children's literature    2122
Mystery                  1401
Thriller                 1392
Crime Fiction            1094
Name: count, dtype: int64

In [11]:
data2.explode("genres").genres.apply(map_genres).value_counts()

genres
Fiction                  5918
Science Fiction          2953
Fantasy                  2594
Children's literature    2122
Mystery                  1401
Thriller                 1392
Crime Fiction            1094
Name: count, dtype: int64

In [12]:
data_ex = data2.explode("genres")
data_ex

,genres,summary
0,Roman à clef,"Old Major, the old boar on the Manor Farm, ca..."
0,Satire,"Old Major, the old boar on the Manor Farm, ca..."
0,Children's literature,"Old Major, the old boar on the Manor Farm, ca..."
0,Speculative fiction,"Old Major, the old boar on the Manor Farm, ca..."
0,Fiction,"Old Major, the old boar on the Manor Farm, ca..."
...,...,...
16555,Thriller,The reader first meets Rapp while he is doing...
16555,Fiction,The reader first meets Rapp while he is doing...
16556,Autobiography,The book follows very rough chronological ord...
16558,Epistolary novel,Makar Devushkin and Varvara Dobroselova are s...


In [13]:
data_ex["genres"] = data_ex.genres.apply(map_genres)
data_ex = data_ex.dropna().drop_duplicates(subset="summary")

In [14]:
data_ex.genres.value_counts()

genres
Science Fiction          2743
Fiction                  2260
Fantasy                  1459
Children's literature    1429
Mystery                   748
Crime Fiction             732
Thriller                  619
Name: count, dtype: int64

In [15]:
data_ex.to_csv("../data/booksFiltered.csv", index=False)